# Advent of Code 2020

In [1]:
from csv import reader
from functools import reduce
from json import dump
from pprint import pprint
from re import match, split, sub

## Day 1

In [2]:
# input formatting
expense_report = [int(n[0]) for n in reader(open("d01/d01.txt"))]

In [3]:
dump(expense_report, open("d01/d01.json", "wt"), indent=1)

In [4]:
# 1-1
def day1_1():
    for i in expense_report:
        if 2020-i in expense_report:
            return(i*(2020-i))

print(day1_1())

618144


In [5]:
# 1-2
def day1_2():
    for i in expense_report:
        for j in expense_report:
            if 2020-i-j in expense_report and i != j:
                return(i*j*(2020-i-j))

print(day1_2())

173538720


## Day 2

Previous input formatting (OS-dependent as it requires GNU sed tool):

```bash
sed -e 's/://g' -e 's/[ -]/,/g' d02/d02.txt > d02/d02.csv
```

```python
password_entries = list(DictReader(open("d02/d02.csv"), fieldnames=['min', 'max', 'char', 'pass']))
for entry in password_entries:
    entry["min"] = int(entry["min"])
    entry["max"] = int(entry["max"])
```

In [6]:
# input formatting
password_entries = []
with open("d02/d02.txt") as fl:
    for raw_line in fl.read().strip().split("\n"):
        raw_line = raw_line.replace(':', '')
        raw_line = raw_line.replace(' ', ',')
        raw_line = raw_line.replace('-', ',')
        line = raw_line.split(',')
        password_entries.append({
            "min": int(line[0]),
            "max": int(line[1]),
            "char": line[2],
            "pass": line[3]
        })

In [7]:
dump(password_entries, open("d02/d02.json", "wt"), indent=1)

In [8]:
# 2-1
len(list(filter(lambda x: x["min"] <= x["pass"].count(x["char"]) <= x["max"],
                password_entries)))

546

In [9]:
# 2-2
def is_password_valid(entry):
    valid_positions = set((entry["min"], entry["max"]))
    actual_positions = set((i+1
                            for i in range(0, len(entry["pass"]))
                            if entry["pass"][i] == entry["char"]))
    return(len(valid_positions.intersection(actual_positions)) == 1)

len(list(filter(lambda x: is_password_valid(x), password_entries)))

275

## Day 3

In [10]:
# input formatting
biome = []

with open("d03/d03.txt") as biome_file:
    for line in biome_file.read().split("\n"):
        biome.append(line*100)  # TODO dirty trick
    biome.pop()

In [11]:
dump(biome, open("d03/d03.json", "wt"), indent=1)

In [12]:
# 3-1
trees_in_path = 0

for l in range(1, len(biome)):
    if biome[l][l*3] == '#':
        trees_in_path += 1

trees_in_path

159

In [13]:
# 3-2
def check_slopes(biome, move_right, move_down):
    trees_in_path = 0

    for x in range(move_down, len(biome), move_down):
        if biome[x][x//move_down*move_right] == '#':
            trees_in_path += 1

    return trees_in_path

In [14]:
check_slopes(biome, 1, 1)   \
* check_slopes(biome, 3, 1) \
* check_slopes(biome, 5, 1) \
* check_slopes(biome, 7, 1) \
* check_slopes(biome, 1, 2)  # 55

6419669520

## Day 4

In [15]:
# input formatting
passports = []
for raw_passport in map(lambda x: split('\s', x ),
                        split('\n\n', open("d04/d04.txt").read().strip())):
    passports.append({
        k: v
        for k, v in (field_string.split(':')
                     for field_string in raw_passport)
    })

In [16]:
dump(passports, open("d04/d04.json", "wt"), indent=1)

In [17]:
# 4-1
len([p for p in passports
     if len(p) == 8
     or (len(p) == 7 and 'cid' not in p)])

237

In [18]:
# 4-2
def is_hgt_valid(h):
    if h[-2:] == 'cm':
        return 150 <= int(h.replace('cm', '')) <= 193
    elif h[-2:] == 'in':
        return 59 <= int(h.replace('in', '')) <= 76
    else:
        return False

len([p for p in passports
     if (len(p) == 8 or (len(p) == 7 and 'cid' not in p))  # same as 4-1
     and 1920 <= int(p["byr"]) <= 2002
     and 2010 <= int(p["iyr"]) <= 2020
     and 2020 <= int(p["eyr"]) <= 2030
     and is_hgt_valid(p["hgt"])
     and match("^#[0-9a-f]{6}$", p["hcl"])
     and p["ecl"] in {"amb", "blu", "brn", "gry", "grn", "hzl", "oth"}
     and match("^[0-9]{9}$", p["pid"])])

172

## Day 5

In [19]:
# input formatting
def row_reducer(row_string):
    row_number = 0
    for i in range(0, len(row_string)):
        row_number += 2**(len(row_string)-i-1) if row_string[i] == "B" else 0
    return row_number//8

def column_reducer(column_string):
    column_number = 0
    for i in range(0, len(column_string)):
        column_number += 2**(len(column_string)-i-1) if column_string[i] == "R" else 0
    return column_number

seats = []
for line in open("d05/d05.txt").read().strip().split():
    seats.append({
        "row": row_reducer(line),
        "column": column_reducer(line),
        "id": row_reducer(line)*8 + column_reducer(line)
    })

In [20]:
dump(seats, open("d05/d05.json", "wt"), indent=1)

In [21]:
# 5-1
max((s["id"] for s in seats))

835

In [22]:
# 5-2
seat_ids = [s["id"] for s in seats]
seat_ids.sort()

for sid in range(seat_ids[0], seat_ids[-1]):
    if (sid not in seat_ids
            and sid-1 in seat_ids
            and sid+1 in seat_ids):
        print(sid)

649


## Day 6

In [23]:
# input formatting
grouped_form_answers = split('\n\n', open("d06/d06.txt").read().strip())

In [24]:
# 6-1
reduce(lambda x, y: x+y,
       map(lambda group_answer: len(set(group_answer.replace("\n", ""))),
           grouped_form_answers))

6587

In [25]:
# 6-2
grouped_yes = []

for group_answer in grouped_form_answers:
    group_answer_by_person = group_answer.split("\n")
    everyone_answered_yes = 0

    for question_letter in set(group_answer.replace("\n", "")):
        if (len(list(filter(lambda person_answer: question_letter in person_answer, group_answer_by_person)))
                == len(group_answer_by_person)):
            everyone_answered_yes += 1

    grouped_yes.append(everyone_answered_yes)

In [26]:
reduce(lambda x, y: x+y, grouped_yes)

3235

## Day 7

### 7-1

In [27]:
# input formatting
rules = {}

for rule_line in open("d07/d07.txt").read().strip().split("\n"):
    split_rule = split('bags?', sub('contain|,|\.|\d', '', rule_line).strip())[:-1]
    rules[split_rule[0].strip()] = [item.strip() for item in split_rule[1:]]

In [28]:
def update_with_nested_bags(color_set, rule_dict, previous_size):
    new_colors = []
    
    for color in color_set:
        for k, v in rule_dict.items():
            if color in v:
                new_colors.append(k)

    color_set.update(new_colors)

    if len(color_set) == previous_size:
        return color_set
    else:
        return update_with_nested_bags(color_set, rule_dict, len(color_set))

len(update_with_nested_bags({"shiny gold"}, rules, 1).difference({"shiny gold", "no other"}))

246

### 7-2

In [29]:
# input formatting
rules = {}

for rule_line in open("d07/d07.txt").read().strip().split("\n"):
    split_rule = [
        rule_sub_item.strip()
        for rule_sub_item in split('bags?', sub('contain|,|\.', '', rule_line).strip())[:-1]
    ]
    rules[split_rule[0]] = {
        reduce(lambda x, y: x+" "+y, rule_sub_item.split(" ")[1:]): int(rule_sub_item.split(" ")[0])
        for rule_sub_item in split_rule[1:]
        if rule_sub_item != "no other"
    }